# LEADING Boot-Camp: Supervised Learning



## 1. Regression example

In [1]:
# load in data
import pandas as pd
housing = pd.read_csv('./data/housing.csv')

In [2]:
housing.head(5)

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY

In [3]:
# split data
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [4]:
# separate the predictors and the labels
x_train = train_set.drop("median_house_value", axis=1) # drop labels for training set
y_train = train_set["median_house_value"].copy()

In [5]:
# imputer, strategy: median
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

In [6]:
# build pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


housing_num = x_train.drop("ocean_proximity", axis=1)

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

x_train_transformed = full_pipeline.fit_transform(x_train)

In [7]:
# separate the predictors and the labels
x_test = test_set.drop("median_house_value", axis=1) 
y_test = test_set["median_house_value"].copy()

# transform the test set
x_test_transformed = full_pipeline.transform(x_test)

In [8]:
# train with regression algorithms
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR

In [9]:
LR = LinearRegression()
DT = DecisionTreeRegressor(random_state=42)
RF = RandomForestRegressor(random_state=42)
KNN = KNeighborsRegressor()
SVM = LinearSVR(random_state=42)

LR.fit(x_train_transformed, y_train)
DT.fit(x_train_transformed, y_train)
RF.fit(x_train_transformed, y_train)
KNN.fit(x_train_transformed, y_train)
SVM.fit(x_train_transformed, y_train)

LinearSVR(random_state=42)

In [10]:
from sklearn.metrics import mean_squared_error
import numpy as np

# evaluate performance on test set
print('RMSE of Linear Regressor: %.3f' % np.sqrt(mean_squared_error(y_test, LR.predict(x_test_transformed))))
print('RMSE of Decision Tree Regressor: %.3f' % np.sqrt(mean_squared_error(y_test, DT.predict(x_test_transformed))))
print('RMSE of Random Forest Regressor: %.3f' % np.sqrt(mean_squared_error(y_test, RF.predict(x_test_transformed))))
print('RMSE of Nearest Neighbor Regressor: %.3f' % np.sqrt(mean_squared_error(y_test, KNN.predict(x_test_transformed))))
print('RMSE of Support Vecotr Machine Regressor: %.3f' % np.sqrt(mean_squared_error(y_test, SVM.predict(x_test_transformed))))

RMSE of Linear Regressor: 70059.193
RMSE of Decision Tree Regressor: 69175.769
RMSE of Random Forest Regressor: 48941.700
RMSE of Nearest Neighbor Regressor: 61309.752
RMSE of Support Vecotr Machine Regressor: 216450.907


## 2. Classification example


In [11]:
titanic = pd.read_csv('./data/titanic.csv')

In [12]:
titanic.head(5)

Survived  Pclass                                               Name  \
0         0       3                             Mr. Owen Harris Braund   
1         1       1  Mrs. John Bradley (Florence Briggs Thayer) Cum...   
2         1       3                              Miss. Laina Heikkinen   
3         1       1        Mrs. Jacques Heath (Lily May Peel) Futrelle   
4         0       3                            Mr. William Henry Allen   

      Sex   Age  Siblings/Spouses Aboard  Parents/Children Aboard     Fare  
0    male  22.0                        1                        0   7.2500  
1  female  38.0                        1                        0  71.2833  
2  female  26.0                        0                        0   7.9250  
3  female  35.0                        1                        0  53.1000  
4    male  35.0                        0                        0   8.0500

In [13]:
# pre-processing
titanic['Sex'][titanic['Sex'] == 'male'] = 0
titanic['Sex'][titanic['Sex'] == 'female'] = 1

y = titanic['Survived']
x = titanic[['Pclass', "Sex", "Age", "Siblings/Spouses Aboard", "Parents/Children Aboard", "Fare"]]

# data splits
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

<ipython-input-13-87bd4d6c93a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic['Sex'][titanic['Sex'] == 'male'] = 0
<ipython-input-13-87bd4d6c93a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic['Sex'][titanic['Sex'] == 'female'] = 1


In [14]:
scalar = StandardScaler()
x_train_scaled = scalar.fit_transform(x_train)
x_test_scaled = scalar.transform(x_test)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [16]:
LR = LogisticRegression(random_state=42)
KNN = KNeighborsClassifier()
NB = GaussianNB()
SVM = LinearSVC(random_state=42)
DT = DecisionTreeClassifier(random_state=42)
RF = RandomForestClassifier(random_state=42)

In [17]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

classifiers = [("Logistic Regression", LR),
               ("Nearest Neighbor", KNN),
               ("Naïve Bayes", NB),
               ("SVM", SVM), 
               ("Decision Tree", DT),
               ("Random Forest", RF)]

for name, classifier in classifiers:
    classifier.fit(x_train_scaled, y_train)
    predictions = classifier.predict(x_test_scaled)

    print("For "+ name +", accuracy, precision, recall, and F1 were:")
    print(accuracy_score(y_test, predictions))
    print(precision_score(y_test, predictions))
    print(recall_score(y_test, predictions))
    print(f1_score(y_test, predictions))  
    print("")

For Logistic Regression, accuracy, precision, recall, and F1 were:
0.7471910112359551
0.7115384615384616
0.5522388059701493
0.6218487394957983

For Nearest Neighbor, accuracy, precision, recall, and F1 were:
0.7808988764044944
0.7121212121212122
0.7014925373134329
0.7067669172932332

For Naïve Bayes, accuracy, precision, recall, and F1 were:
0.7359550561797753
0.6923076923076923
0.5373134328358209
0.6050420168067226



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


For SVM, accuracy, precision, recall, and F1 were:
0.7359550561797753
0.7
0.5223880597014925
0.5982905982905983

For Decision Tree, accuracy, precision, recall, and F1 were:
0.7471910112359551
0.6486486486486487
0.7164179104477612
0.6808510638297873

For Random Forest, accuracy, precision, recall, and F1 were:
0.7584269662921348
0.6818181818181818
0.6716417910447762
0.6766917293233083



In [18]:
accuracy_score(y_train, DT.predict(x_train_scaled))

0.9858956276445698

## 3. Cross-validation
Now, since we're doing the training/test fit, there's always the unfortunate chance that we're overfit to the specific data in the training set, and/or reality of the test set. To alleviate this issue it's not uncommon to _cross-validate_ an experiment. Here, a train vs. test split is still done, but afterwords the training set is split again into *$k$ folds*, i.e., samples of equal size. After the folds are drawn, each of the $k$ pairings of $1$ _validation_ and $k-1$ _training_ records are used to train and evaluate a model. In other words, if there are $k=5$ folds ${A,B,C,D,E}$, the $k$ validation pairs are:

1. train: $\{B,C,D,E\}$; validate: $\{A\}$
2. train: $\{A,C,D,E\}$; validate: $\{B\}$
3. train: $\{A,B,D,E\}$; validate: $\{C\}$
4. train: $\{A,B,C,E\}$; validate: $\{D\}$
5. train: $\{A,B,C,D\}$; validate: $\{E\}$

For each pair, there are resulting coefficients. Here's a good graphic to visualize the process:

![K Folds](./images/k-folds.png)

In [19]:
from sklearn.model_selection import cross_val_score

DT = DecisionTreeClassifier(random_state=42)
print(cross_val_score(DT, x_train_scaled, y_train, cv=5, scoring = 'accuracy'))

[0.73239437 0.77464789 0.83098592 0.73943662 0.80141844]
